In [1]:
import numpy as np
import random
import pickle


In [2]:
# !pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("recommender")\
    .getOrCreate()
sc = spark.sparkContext

22/12/18 08:33:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
data = sc.textFile("../data/ml-latest-small/ratings.csv")
originRDD = data.map(
  lambda l: l.split(',')
)
head = originRDD.first()
originRDD = originRDD.filter(lambda x: x!= head)\
.map(
  lambda l: (int(l[0]), int(l[1]), float(l[2]))
)


In [7]:
trainRDD, testRDD = originRDD.randomSplit([0.8,0.2], 42)

In [9]:
from recommenders.self_recommender import DSGD, ALS

In [7]:
# model = ALS(num_factor=10, max_iter=10, lambd=10)
# model.train(sc, originRDD, trainRDD, testRDD)
# model.plot('ALS nfactor=10, lambda=10')

In [10]:
SGDmodel = DSGD(step_size=0.01, num_factor=10, max_iter=10, lambd=10)
SGDmodel.train(sc, originRDD, trainRDD, testRDD)
SGDmodel.plot_rmse('DSGD stepsize=1 nfactor=10 lambda=10')
print(SGDmodel.get_test_rmse())

/home/noing/git/recommender_implement_with_spark/experiment/recommenders/self_recommender.py:51: RuntimeWarning: overflow encountered in add
  sse = R.map(lambda x: (x[2] - w.value[x[0]].dot(h.value[x[1]].T))**2 ).reduce(lambda x,y: x+y)


22/12/18 08:47:39 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 666, in main
    eval_type = read_int(infile)
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:765)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1211)
	a

22/12/18 08:47:39 WARN TaskSetManager: Lost task 5.0 in stage 17.0 (TID 34) (192.168.1.16 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/pyspark/rdd.py", line 2869, in <lambda>
    return self.map(lambda x: (f(x), x))
  File "/home/noing/git/recomme

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 17.0 failed 1 times, most recent failure: Lost task 5.0 in stage 17.0 (TID 34) (192.168.1.16 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/noing/data_framework/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*ar

In [9]:
# print(SGDmodel.get_test_rmse())
# print(SGDmodel.get_train_rmse())

#RUN ON MULTIPLE HYPER PARAMETER

In [10]:
max_iters = [10, 20, 50]
lambds = [0.1, 0.5, 10]
stepsizes = [0.001, 0.005, 0.0025]
num_factors = [10, 20, 30]

RUN SGD HYPER PARAMETER

In [11]:
train_DSGD_result_list={}
test_DSGD_result_list={}
time_cost_DSGD_result_list={}

In [12]:
max_iter = 10
for lamb in lambds:
    for stepsize in stepsizes:
        for num_factor in num_factors:
            model = DSGD(num_factor=num_factor, step_size=stepsize, max_iter=max_iter, lambd=lamb)
            model.train(sc, originRDD, trainRDD, testRDD)
            train_DSGD_result_list[f'DSGD stepsize={stepsize}, miter={max_iter}, nfactor={num_factor}, lambda={lamb}']=model.get_train_rmse()
            test_DSGD_result_list[f'DSGD stepsize={stepsize}, miter={max_iter}, nfactor={num_factor}, lambda={lamb}']=model.get_test_rmse()
            time_cost_DSGD_result_list[f'DSGD stepsize={stepsize}, miter={max_iter}, nfactor={num_factor}, lambda={lamb}']=model.get_time_cost()
            model.save_plot_rmse(f'DSGD stepsize={stepsize}, miter={max_iter}, nfactor={num_factor}, lambda={lamb}')

/home/noing/git/recommender_implement_with_spark/self_recommender.py:51: RuntimeWarning: overflow encountered in add
  sse = R.map(lambda x: (x[2] - w.value[x[0]].dot(h.value[x[1]].T))**2 ).reduce(lambda x,y: x+y)
/home/noing/git/recommender_implement_with_spark/self_recommender.py:51: RuntimeWarning: overflow encountered in add
  sse = R.map(lambda x: (x[2] - w.value[x[0]].dot(h.value[x[1]].T))**2 ).reduce(lambda x,y: x+y)
/home/noing/git/recommender_implement_with_spark/self_recommender.py:51: RuntimeWarning: overflow encountered in add
  sse = R.map(lambda x: (x[2] - w.value[x[0]].dot(h.value[x[1]].T))**2 ).reduce(lambda x,y: x+y)
/home/noing/git/recommender_implement_with_spark/self_recommender.py:51: RuntimeWarning: overflow encountered in add
  sse = R.map(lambda x: (x[2] - w.value[x[0]].dot(h.value[x[1]].T))**2 ).reduce(lambda x,y: x+y)
/home/noing/git/recommender_implement_with_spark/self_recommender.py:51: RuntimeWarning: overflow encountered in add
  sse = R.map(lambda x: (x[

In [13]:
with open('test_rmse_DSGD.pkl', 'wb') as f:
    pickle.dump(test_DSGD_result_list, f)
with open('train_rmse_DSGD.pkl', 'wb') as f:
    pickle.dump(train_DSGD_result_list, f)
with open('time_cost_DSGD.pkl', 'wb') as f:
    pickle.dump(time_cost_DSGD_result_list, f)

In [14]:
test_DSGD_result_list

{'DSGD stepsize=0.001, miter=10, nfactor=10, lambda=0.1': 0.9589843,
 'DSGD stepsize=0.001, miter=10, nfactor=20, lambda=0.1': 1.0023545,
 'DSGD stepsize=0.001, miter=10, nfactor=30, lambda=0.1': 1.2197591,
 'DSGD stepsize=0.005, miter=10, nfactor=10, lambda=0.1': 0.8692193,
 'DSGD stepsize=0.005, miter=10, nfactor=20, lambda=0.1': 0.9045088,
 'DSGD stepsize=0.005, miter=10, nfactor=30, lambda=0.1': 1.0031549,
 'DSGD stepsize=0.0025, miter=10, nfactor=10, lambda=0.1': 0.88545495,
 'DSGD stepsize=0.0025, miter=10, nfactor=20, lambda=0.1': 0.93783075,
 'DSGD stepsize=0.0025, miter=10, nfactor=30, lambda=0.1': 1.0546317,
 'DSGD stepsize=0.001, miter=10, nfactor=10, lambda=0.5': 0.96232575,
 'DSGD stepsize=0.001, miter=10, nfactor=20, lambda=0.5': 0.9995485,
 'DSGD stepsize=0.001, miter=10, nfactor=30, lambda=0.5': 1.2177838,
 'DSGD stepsize=0.005, miter=10, nfactor=10, lambda=0.5': 0.8818223,
 'DSGD stepsize=0.005, miter=10, nfactor=20, lambda=0.5': 0.9082716,
 'DSGD stepsize=0.005, miter

In [15]:
time_cost_DSGD_result_list

{'DSGD stepsize=0.001, miter=10, nfactor=10, lambda=0.1': datetime.timedelta(seconds=103, microseconds=44407),
 'DSGD stepsize=0.001, miter=10, nfactor=20, lambda=0.1': datetime.timedelta(seconds=94, microseconds=781555),
 'DSGD stepsize=0.001, miter=10, nfactor=30, lambda=0.1': datetime.timedelta(seconds=100, microseconds=210765),
 'DSGD stepsize=0.005, miter=10, nfactor=10, lambda=0.1': datetime.timedelta(seconds=97, microseconds=151655),
 'DSGD stepsize=0.005, miter=10, nfactor=20, lambda=0.1': datetime.timedelta(seconds=99, microseconds=249261),
 'DSGD stepsize=0.005, miter=10, nfactor=30, lambda=0.1': datetime.timedelta(seconds=100, microseconds=42229),
 'DSGD stepsize=0.0025, miter=10, nfactor=10, lambda=0.1': datetime.timedelta(seconds=91, microseconds=650134),
 'DSGD stepsize=0.0025, miter=10, nfactor=20, lambda=0.1': datetime.timedelta(seconds=93, microseconds=876866),
 'DSGD stepsize=0.0025, miter=10, nfactor=30, lambda=0.1': datetime.timedelta(seconds=94, microseconds=227252

In [16]:
train_DSGD_result_list

{'DSGD stepsize=0.001, miter=10, nfactor=10, lambda=0.1': 0.6738309229418369,
 'DSGD stepsize=0.001, miter=10, nfactor=20, lambda=0.1': 0.7280036114778541,
 'DSGD stepsize=0.001, miter=10, nfactor=30, lambda=0.1': inf,
 'DSGD stepsize=0.005, miter=10, nfactor=10, lambda=0.1': 0.6350491730760115,
 'DSGD stepsize=0.005, miter=10, nfactor=20, lambda=0.1': 0.6438002632387755,
 'DSGD stepsize=0.005, miter=10, nfactor=30, lambda=0.1': 0.6885002721247431,
 'DSGD stepsize=0.0025, miter=10, nfactor=10, lambda=0.1': 0.6465879608428324,
 'DSGD stepsize=0.0025, miter=10, nfactor=20, lambda=0.1': 0.6669746900679041,
 'DSGD stepsize=0.0025, miter=10, nfactor=30, lambda=0.1': 0.7312903262081112,
 'DSGD stepsize=0.001, miter=10, nfactor=10, lambda=0.5': 0.6708585620769911,
 'DSGD stepsize=0.001, miter=10, nfactor=20, lambda=0.5': 0.7299226602902844,
 'DSGD stepsize=0.001, miter=10, nfactor=30, lambda=0.5': 0.9003176144282036,
 'DSGD stepsize=0.005, miter=10, nfactor=10, lambda=0.5': 0.6428683443957439

In [17]:
train_ALS_result_list = {}
test_ALS_result_list = {}
time_cost_ALS_result_list = {}

In [18]:
max_iters = [10, 20, 50]
lambds = [10, 20, 50]
num_factors = [10, 20, 30]

In [20]:
max_iter = 10
for lamb in lambds:
    for num_factor in num_factors:
        model = ALS(num_factor=num_factor, max_iter=max_iter, lambd=lamb)
        model.train(sc, originRDD, trainRDD, testRDD)
        train_ALS_result_list[f'ALS miter={max_iter}, nfactor={num_factor}, lambda={lamb}']=model.get_train_rmse()
        test_ALS_result_list[f'ALS miter={max_iter}, nfactor={num_factor}, lambda={lamb}']=model.get_test_rmse()
        time_cost_ALS_result_list[f'ALS miter={max_iter}, nfactor={num_factor}, lambda={lamb}']=model.get_time_cost()
        model.save_plot_rmse(f'ALS miter={max_iter}, nfactor={num_factor}, lambda={lamb}')

In [21]:
with open('test_rmse_ALS.pkl', 'wb') as f:
    pickle.dump(test_ALS_result_list, f)
with open('train_rmse_ALS.pkl', 'wb') as f:
    pickle.dump(train_ALS_result_list, f)
with open('time_cost_ALS.pkl', 'wb') as f:
    pickle.dump(time_cost_ALS_result_list, f)

In [22]:
test_ALS_result_list

{'ALS miter=10, nfactor=10, lambda=10': 0.9570722942299743,
 'ALS miter=10, nfactor=20, lambda=10': 0.9764623469334455,
 'ALS miter=10, nfactor=30, lambda=10': 0.9832462796758047,
 'ALS miter=10, nfactor=10, lambda=20': 1.028198216429219,
 'ALS miter=10, nfactor=20, lambda=20': 1.0309498130080652,
 'ALS miter=10, nfactor=30, lambda=20': 1.0326945035583546,
 'ALS miter=10, nfactor=10, lambda=50': 1.296582742545248,
 'ALS miter=10, nfactor=20, lambda=50': 1.2974624662747276,
 'ALS miter=10, nfactor=30, lambda=50': 1.2978592312694548}

In [23]:
train_ALS_result_list

{'ALS miter=10, nfactor=10, lambda=10': 0.8107581983573895,
 'ALS miter=10, nfactor=20, lambda=10': 0.7417343304705022,
 'ALS miter=10, nfactor=30, lambda=10': 0.698869811648173,
 'ALS miter=10, nfactor=10, lambda=20': 0.9523750897127452,
 'ALS miter=10, nfactor=20, lambda=20': 0.9471723765343775,
 'ALS miter=10, nfactor=30, lambda=20': 0.945967209858048,
 'ALS miter=10, nfactor=10, lambda=50': 1.2555450696293406,
 'ALS miter=10, nfactor=20, lambda=50': 1.2565166173489608,
 'ALS miter=10, nfactor=30, lambda=50': 1.256935554128971}

In [24]:
time_cost_ALS_result_list

{'ALS miter=10, nfactor=10, lambda=10': datetime.timedelta(seconds=49, microseconds=634605),
 'ALS miter=10, nfactor=20, lambda=10': datetime.timedelta(seconds=46, microseconds=949738),
 'ALS miter=10, nfactor=30, lambda=10': datetime.timedelta(seconds=47, microseconds=396995),
 'ALS miter=10, nfactor=10, lambda=20': datetime.timedelta(seconds=45, microseconds=160740),
 'ALS miter=10, nfactor=20, lambda=20': datetime.timedelta(seconds=47, microseconds=155623),
 'ALS miter=10, nfactor=30, lambda=20': datetime.timedelta(seconds=47, microseconds=714878),
 'ALS miter=10, nfactor=10, lambda=50': datetime.timedelta(seconds=45, microseconds=714528),
 'ALS miter=10, nfactor=20, lambda=50': datetime.timedelta(seconds=42, microseconds=438291),
 'ALS miter=10, nfactor=30, lambda=50': datetime.timedelta(seconds=43, microseconds=673990)}